In [1]:
# Uncomment and run the appropriate command for your operating system, if required
# No installation is reqiured on Google Colab / Kaggle notebooks

# Linux / Binder / Windows (No GPU)
# !pip install numpy matplotlib torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Linux / Windows (GPU)
# pip install numpy matplotlib torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
 
# MacOS (NO GPU)
# !pip install numpy matplotlib torch torchvision torchaudio

# Installing necessary packages

In [8]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

required_packages = ['numpy', 'matplotlib', 'torch', 'torchvision', 'torchaudio', 'transformers', 'pillow']

for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        install(package)

In [9]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils
from PIL import Image
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt

# Data Preprocessing

In [12]:
import re
from PIL import UnidentifiedImageError

image_dir = 'dataset/images'
files = os.listdir(image_dir)  # list of all files in the directory

counter = 1

# pattern to match files already in the `artwork\d{3}\.\w+` format
pattern = re.compile(r'artwork\d{3}\.\w+')

for file_name in files:
    file_ext = os.path.splitext(file_name)[1].lower()
    
    if not pattern.match(file_name) and file_ext != '.jpg':
        new_file_name = f'artwork{counter:03d}.jpg'
        
        old_file_path = os.path.join(image_dir, file_name)
        new_file_path = os.path.join(image_dir, new_file_name)
        
        try:
            # Open the image, convert to RGB, resize, and save as JPG
            with Image.open(old_file_path) as img:
                img = img.convert('RGB')  # Convert to RGB
                img = img.resize((128, 128))  # Resize to 128x128
                img.save(new_file_path, 'JPEG')
            counter += 1
        except UnidentifiedImageError:
            print(f"Cannot identify image file '{file_name}', skipping.")
        
print("Renaming and conversion completed.")

Cannot identify image file 'artwork001', skipping.
Cannot identify image file '.DS_Store', skipping.
Cannot identify image file 'artwork002', skipping.
Renaming and conversion completed.


In [5]:
from PIL import UnidentifiedImageError

# custom dataset
class TextImageDataset(Dataset):
    def __init__(self, text_dir, image_dir, transform=None):
        self.text_dir = text_dir
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = sorted(os.listdir(image_dir))
        self.text_files = sorted(os.listdir(text_dir))
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.valid_image_files = self.filter_valid_images()

    def filter_valid_images(self):
        valid_files = []
        for img_name in self.image_files:
            img_path = os.path.join(self.image_dir, img_name)
            try:
                img = Image.open(img_path)
                img.verify()  # Verify that it is, in fact, an image
                valid_files.append(img_name)
            except (UnidentifiedImageError, IOError):
                print(f"Invalid image file: {img_path}")
        return valid_files

    def __len__(self):
        return len(self.valid_image_files)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.valid_image_files[idx])
        image = Image.open(img_name)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        txt_name = os.path.join(self.text_dir, self.text_files[idx])
        description = self.read_text_file(txt_name)
        
        tokens = self.tokenizer(description, return_tensors='pt', padding='max_length', truncation=True, max_length=128)
        return image, tokens['input_ids'].squeeze(), tokens['attention_mask'].squeeze()

    def read_text_file(self, file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        except UnicodeDecodeError:
            with open(file_path, 'r', encoding='latin1') as f:
                return f.read()

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize expects 3 channels (RGB)
])

# creating dataset and dataloader
dataset = TextImageDataset(text_dir='dataset/text', image_dir='dataset/images', transform=transform)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

Invalid image file: dataset/images/artwork001
Invalid image file: dataset/images/artwork002
Invalid image file: dataset/images/artwork070.jxl


# Feature Engineering

In [6]:
# defining the Generator Network
class Generator(nn.Module):
    def __init__(self, latent_dim, text_dim, img_channels):
        super(Generator, self).__init__()
        self.text_embedding = nn.Linear(text_dim, latent_dim)
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, img_channels, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, noise, text):
        text = text.float()
        text_embedding = self.text_embedding(text)
        x = torch.cat([noise, text_embedding], dim=1)
        x = x.unsqueeze(2).unsqueeze(3)
        return self.gen(x)

# defining the Discriminator Network
class Discriminator(nn.Module):
    def __init__(self, img_channels, text_dim, hidden_dim, img_size=64):
        super(Discriminator, self).__init__()
        
        # Calculate the size of the feature maps after each convolution layer
        def conv_output_size(size, kernel_size=4, stride=2, padding=1):
            return (size - kernel_size + 2 * padding) // stride + 1

        self.img_size = img_size
        self.hidden_dim = hidden_dim

        current_size = img_size
        self.img_dis = nn.Sequential(
            nn.Conv2d(img_channels, hidden_dim // 4, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True)
        )
        current_size = conv_output_size(current_size)

        self.img_dis.add_module("conv2", nn.Conv2d(hidden_dim // 4, hidden_dim // 2, 4, stride=2, padding=1))
        self.img_dis.add_module("lrelu2", nn.LeakyReLU(0.2, inplace=True))
        current_size = conv_output_size(current_size)

        self.img_dis.add_module("conv3", nn.Conv2d(hidden_dim // 2, hidden_dim, 4, stride=2, padding=1))
        self.img_dis.add_module("lrelu3", nn.LeakyReLU(0.2, inplace=True))
        current_size = conv_output_size(current_size)

        self.text_embedding = nn.Linear(text_dim, hidden_dim)
        self.final = nn.Linear(hidden_dim * (current_size * current_size) + hidden_dim, 1)  # dynamically calculated

    def forward(self, img, text):
        img_out = self.img_dis(img).view(img.size(0), -1)  # flatten the output
        text_embedding = self.text_embedding(text.float())  # ensure text is float
        x = torch.cat([img_out, text_embedding], dim=1)
        return self.final(x)

text_dim = 768  # BERT base model output dimension
latent_dim = 100
hidden_dim = 512  # to be adjusted as needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator = Generator(latent_dim, text_dim, 3).to(device)
discriminator = Discriminator(3, text_dim, hidden_dim).to(device)

# Training the Model
The generator and discriminator are trained here using a loop.
## Optimization
The optimization is embedded within the training loop. The code uses Adam optimizer to update the weights of both the generator and the discriminator.
## Evaluation
Evaluation happens within the training loop where loss values for both the generator and the discriminator are printed every 100 steps. Additionally, generated sample images are saved at the end of each epoch to visualize the generator’s progress.

In [7]:
def train_gan(generator, discriminator, dataloader, epochs, lr, device):
    criterion = nn.BCEWithLogitsLoss()
    optim_g = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optim_d = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

    for epoch in range(epochs):
        for i, (images, text_ids, text_mask) in enumerate(dataloader):
            batch_size = images.size(0)
            real_labels = torch.ones(batch_size, 1).to(device)  # Define real labels
            fake_labels = torch.zeros(batch_size, 1).to(device)  # Define fake labels

            images = images.to(device)
            text_ids = text_ids.to(device)
            text_mask = text_mask.to(device)

            optim_d.zero_grad()
            outputs = discriminator(images, text_ids)
            real_loss = criterion(outputs, real_labels)
            real_loss.backward()

            noise = torch.randn(batch_size, latent_dim).to(device)
            fake_images = generator(noise, text_ids)
            outputs = discriminator(fake_images.detach(), text_ids)
            fake_loss = criterion(outputs, fake_labels)
            fake_loss.backward()
            optim_d.step()

            optim_g.zero_grad()
            outputs = discriminator(fake_images, text_ids)
            g_loss = criterion(outputs, real_labels)
            g_loss.backward()
            optim_g.step()

            if (i + 1) % 100 == 0:
                print(f'Epoch [{epoch + 1}/{epochs}], Step [{i + 1}/{len(dataloader)}], '
                      f'D Loss: {real_loss.item() + fake_loss.item():.4f}, G Loss: {g_loss.item():.4f}')

        with torch.no_grad():
            sample_noise = torch.randn(batch_size, latent_dim).to(device)
            sample_images = generator(sample_noise, text_ids)
            os.makedirs('samples', exist_ok=True)
            utils.save_image(sample_images, f'samples/sample_epoch_{epoch + 1}.png', nrow=8, normalize=True)

    torch.save(generator.state_dict(), 'generator.pth')
    torch.save(discriminator.state_dict(), 'discriminator.pth')

# parameter setting
image_size = 64  
batch_size = 128
epochs = 25
lr = 0.0002

# adjusting Initialization
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
text_dim = 128  # BERT base model output dimension
latent_dim = 100  # latent vector dimension
hidden_dim = 128  # adjusted hidden dimension

generator = Generator(latent_dim, text_dim, 3).to(device)
discriminator = Discriminator(3, text_dim, hidden_dim).to(device)

# training the models
train_gan(generator, discriminator, dataloader, epochs, lr, device)

RuntimeError: Given transposed=1, weight of size [100, 512, 4, 4], expected input[128, 200, 1, 1] to have 100 channels, but got 200 channels instead